### Treading 모듈
Python 프로그램은 기본적으로 하나의 스레드에서 실행된다. 별도의 실행흐름을 만들기 위해서는 추가적으로 스레드를 생성해야한다.
이때 threading 모듈 (High 레벨) 혹은 thread 모듈 (Low 레벨)을 사용할 수 있다. thread 모듈은 deprecate되어 단독으로 사용되고 있지 않으며, 일반적으로 thread 모듈 위에서 동작하는 threading 모듈을 통해서 스레드를 사용한다.

Treading 모듈을 통해 스레드를 사용하는 방법은 아래의 2가지가 있다.
1. ```threading.Thread()``` 메소드을 사용해, 스레드를 통해 실행하고자 하는 함수와 인자를 전달하고 ```start()``` 메소드로 실행시키는 방법
2. ```threading.Thread``` 객체를 상속받은 클래스의 ```run()``` 메소드를 오버라이딩해서 사용하는 방법

관습적으로 2번의 방법을 많이 사용하지만, 2번 방법은 각 스레드별로 별도의 자원관리를 위한 방법이다. 때문에 스레드별로 네임스페이스를 나누거나 자원을 관리할 필요가 없다면 간단한 1번 방법을 사용하는 것이 좋다.

In [13]:
import threading

def print_msg(msg):
    print("{0} in sub thread : [id {1}]".format(msg, threading.get_ident()))

msg = "Hello world!!!"
print("{0} in main thread : [id {1}]".format(msg, threading.get_ident()))

thread_1 = threading.Thread(target=print_msg, args=(msg, )) # Tread() 메소드를 통해 스레드 생성
thread_1.start() # 생성된 스레드를 실행
thread_1.join() # 실행시킨 스레드가 끝날 때까지 대기

Hello world!!! in main thread : [id 14316]
Hello world!!! in sub thread : [id 23580]


### Thread 동기화
스레드는 stack 영역만 따로 갖기 때문에 전역 변수 등을 공유할 수 있다. 이때 공유되는 하나의 자원에 여러 스레드가 동시에 접근하는 경우 문제가 발생할 수 있는데 이러한 상황을 Race Condition이라고 한다. 그리고 공유하는 자원에 대해 Race Condition 없이 다수의 스레드가 작업을 처리할 수 있다면 이것을 Thread-safe하다고 한다.

아래의 코드는 공유자원 ```x```에 접근하는 스레드 ```t1```과 ```t2```에 대한 접근통제를 하지 않는다.
Thread-safe하지 않기 때문에 공유 자원 ```x```에 대한 연산결과가 실행시 마다 다르게 나타난다.

In [10]:
import threading

x = 0 # A shared value

def foo(amount, value):
    global x

    for i in range(amount):
        x += value

t1 = threading.Thread(target=foo, args=(1000000, 1))
t2 = threading.Thread(target=foo, args=(1000000, -1))
t1.start() 
t2.start()
t1.join()
t2.join()

print("Shared value : {0}".format(x))

Shared value : -48981


Thread 동기화란 공유자원에 접근 하는 스레드를 제한하여 공유자원에 대한 연산이 순차적으로 이루어질 수 있도록 통제하는 것이다.
Threading 모듈은 동기화를 위해 Lock, Condition, Semaphore 등 다양한 기능을 제공한다. 아래는 Lock에 대한 예제이이다.

Lcok은 한 명만 들어갈 수 있는 화장시로 생각하면 편한데, 먼저 온 스레드가 ```Lock.acquire()```를 통해 Lock을 획득한다. 이때 다른 스레드가 Lcok을 소유하고 있다면 Lock을 획득할 수 있을 때까지 대기한다. LocK을 획득한 이후 공유자원에 대한 작업을 수행한 뒤 ```Lock.release()```를 통해 Lock을 반납하여 ```Lock.acquire()```로 대기중인  다른 스레드가 Lock을 획득할 수 있도록 한다.

In [6]:
import threading

x = 0 # A shared value
lock = threading.Lock() # Mutex

def foo(amount, value):
    global x
    lock.acquire() # Mutex를 얻기위해 기다린다. running -> waitting

    for i in range(amount): # Mutex를 얻은 후에 실행된다.
        x += value 

    lock.release() # Mutex를 반납한다.

t1 = threading.Thread(target=foo, args=(1000000, 1))
t2 = threading.Thread(target=foo, args=(1000000, -1))
t1.start()
t2.start()
t1.join()
t2.join()

print("Shared value : {0}".format(x))

Shared value : 0


### Daemon Thread
데몬 스레드는 백그라운드에서 실행되는 스레드로, 파이썬 스크립트는 데몬 스레드만 남게될 경우 바로 종료된다. 즉 메인 스레드 등 데몬이 아닌 스레드가 모두 종료되었고, 현재 데몬 스레드들만 실행 중 이라면 파이썬 스크립트는 종료된다.
```threading.Thread()```로 스레드 생성시 ```daemon=True```로 설정하면 데몬스레드를 만들 수 있다.
```daemon``` 옵션의 기본값은 ```False```임으로 ```True```로 따로 설정하지 않으면 해당 스레드는 작업을 마칠 때까지 계속 실행된다.
<br><small>(IDLE, Jupyer 등 콘솔에서 실행 시 메인 스레드인 콘솔 스레드가 종료되지 않기 때문에 아래의 데몬 스레드는 작업을 마칠 때까지 종료되지 않는다.)</small>


In [31]:
import threading
import time

def func(value, count):
    while count > 0:
        print("{0} : in {1}".format(value, threading.get_ident()))
        time.sleep(1)
        count -= 1
        
t1 = threading.Thread(target=func, args=("Daemon", 5), daemon=True)
t2 = threading.Thread(target=func, args=("Non-Daemon", 2))
t1.start()
t2.start()

Daemon : in 14692
Non-Daemon : in 2352
Daemon : in 14692
Non-Daemon : in 2352
Daemon : in 14692
Daemon : in 14692
Daemon : in 14692
